# Axona to NWB

Convert Axona raw (`.bin` + `.set`) or unit data (`.X`, `.pos`, `.eeg`, `.egf`) to NWB format. 

It might be handy to convert the position data in the `.bin` file to a `.pos` file, allowing us to use the same code for both axona formats, as well as the Intan format later on (requiring the Hussaini lab to create their own `.pos` files based on the position tracking used). 

### Axona raw (`.bin` + `.set`)

In [86]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
import os
from pathlib import Path

# Old .bin file
base_dir = Path('/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/')
dir_name = base_dir / 'example_data_raw'
base_filename = '20201004_Raw'

# New .bin file
base_dir = Path('/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data')
dir_name = base_dir / '06172021-HPC-B6-RAW'
base_filename = '06172021-HPC-B6-RAW'
filename = os.path.join(dir_name, base_filename)
set_file = filename + '.set'
bin_file = filename + '.bin'
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW


In [88]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface, AxonaPositionDataInterface
)

In [89]:
class HussainiAxonaNWBConverter(NWBConverter):
    data_interface_classes = dict(
        AxonaRecordingExtractorInterface=AxonaRecordingExtractorInterface,
        AxonaPositionDataInterface=AxonaPositionDataInterface
    )

In [90]:
import random
import string
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np
import re
import datetime
import json
from jsonschema import validate, ValidationError

from nwb_conversion_tools.utils.json_schema import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [91]:
# Specify source data

source_data = dict(
    AxonaPositionDataInterface=dict(
        filename=set_file
    ),
    AxonaRecordingExtractorInterface=dict(
        filename=set_file
    )
)
print(json.dumps(source_data, indent=2))

{
  "AxonaPositionDataInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.set"
  },
  "AxonaRecordingExtractorInterface": {
    "filename": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.set"
  }
}


In [92]:
# Initialize HussainiAxonaNWBConverter

converter = HussainiAxonaNWBConverter(source_data=source_data)

Source data is valid!


In [93]:
# Get metadata

metadata = converter.get_metadata()
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2021-06-17T12:22:02',
  'identifier': '6ff7dabe-ed6d-45aa-b966-635423873bf1',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.'}],
  'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw',
   'description': 'Raw acquisition traces.'}}}

In [94]:
# Get metadata_schema from converter

metadata_schema = converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": ""
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "379de3e3-3a13-4573-81ec-f0f8dcf5cfd8"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "2021-06-17T12:22:02"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment",
        "default": [
          "Abid"
        ]
      },
      "e

In [95]:
# Validate metadata against metadata_schema

validate(
    instance=converter.get_metadata(),
    schema=converter.get_metadata_schema()
)

In [96]:
converter.get_metadata()

{'NWBFile': {'session_description': '',
  'session_start_time': '2021-06-17T12:22:02',
  'identifier': '1a322ca2-36db-4560-bde9-95ea3d52004a',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.'}],
  'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw',
   'description': 'Raw acquisition traces.'}}}

In [97]:
from nwb_conversion_tools.utils.conversion_tools import (
    get_default_nwbfile_metadata, make_nwbfile_from_metadata
)

In [98]:
metadata

{'NWBFile': {'session_description': '',
  'session_start_time': '2021-06-17T12:22:02',
  'identifier': '6ff7dabe-ed6d-45aa-b966-635423873bf1',
  'experimenter': ['Abid']},
 'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.'}],
  'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw',
   'description': 'Raw acquisition traces.'}}}

In [99]:
output_file = base_dir / 'out_example.nwb'

In [100]:
# Export to NWB file

output_file = base_dir / 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file,
    overwrite=True,
    save_to_file=True,
    conversion_options=None
)

Metadata is valid!


/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/pynwb/file.py:801: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/common/table.py:447: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)
/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'SpatialSeries/timestamps': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


NWB file saved at /mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/out_example.nwb!


In [15]:
# Check NWB file

from pynwb import NWBHDF5IO

fname = output_file
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

OSError: Unable to open file (truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048)

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt


from nwbwidgets import nwb2widget

output_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/nwb/axona_se_MS4.nwb'
#output_file = '/mnt/d/NWB/data/NWBData/NWBData/P9HMH_NOID5.nwb'

io = NWBHDF5IO(output_file, mode='r')
nwb = io.read()

nwb2widget(nwb)

In [17]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import get_position_object

pos = get_position_object(filename=set_file)

In [18]:
nwb.processing['behavior'].data_interfaces['Position'].spatial_series['t'].data

KeyError: 'behavior'

In [ ]:
pos.spatial_series['t'].timestamps

### Parse `.pos` files

In [128]:
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import parse_generic_header

# Old pos file
pos_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/sample_bin_to_tint/axona_sample.pos'

# New pos file
pos_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-UNIT/06172021-HPC-B6-UNIT/06172021-B6-HPC-UNIT.pos'
print(pos_file)

pos_header = parse_generic_header(pos_file, None)
pos_header

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-UNIT/06172021-HPC-B6-UNIT/06172021-B6-HPC-UNIT.pos


{'trial_date': 'Thursday, 17 Jun 2021',
 'trial_time': '12:22:02',
 'experimenter': 'Abid',
 'comments': '',
 'duration': '301',
 'sw_version': '1.2.2.16',
 'num_colours': '4',
 'min_x': '0',
 'max_x': '640',
 'min_y': '0',
 'max_y': '480',
 'window_min_x': '247',
 'window_max_x': '426',
 'window_min_y': '179',
 'window_max_y': '357',
 'timebase': '50 hz',
 'bytes_per_timestamp': '4',
 'sample_rate': '50.0 hz',
 'EEG_samples_per_position': '5',
 'bearing_colour_1': '0',
 'bearing_colour_2': '0',
 'bearing_colour_3': '0',
 'bearing_colour_4': '0',
 'pos_format': 't,x1,y1,x2,y2,numpix1,numpix2',
 'bytes_per_coord': '2',
 'pixels_per_metre': '300',
 'num_pos_samples': '15050'}

Data (with memory map)

In [129]:
def get_header_bstring(file):
    """
    Scan file for the occurrence of 'data_start' and return the header
    as byte string

    Parameters
    ----------
    file (str or path): file to be loaded

    Returns
    -------
    str: header byte content
    """

    header = b''
    with open(file, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                header += b'data_start'
                break
            else:
                header += bin_line
    return header

In [130]:
with open(pos_file, 'rb') as f:
    print(f.read(532))
    print(f.read(1))

b'trial_date Thursday, 17 Jun 2021\r\ntrial_time 12:22:02\r\nexperimenter Abid\r\ncomments \r\nduration 301       \r\nsw_version 1.2.2.16\r\nnum_colours 4\r\nmin_x 0\r\nmax_x 640\r\nmin_y 0\r\nmax_y 480\r\nwindow_min_x 247\r\nwindow_max_x 426\r\nwindow_min_y 179\r\nwindow_max_y 357\r\ntimebase 50 hz\r\nbytes_per_timestamp 4\r\nsample_rate 50.0 hz\r\nEEG_samples_per_position 5\r\nbearing_colour_1 0\r\nbearing_colour_2 0\r\nbearing_colour_3 0\r\nbearing_colour_4 0\r\npos_format t,x1,y1,x2,y2,numpix1,numpix2\r\nbytes_per_coord 2\r\npixels_per_metre 300\r\nnum_pos_samples 15050     \r'
b'\n'


### LFP data

First, let's look at the tools from the Hussaini lab. And let's also remind ourselves what the file formats of .eeg and .egf data should look like:

```
EEG data is usually recorded continuously at 250 Hz in unit recording mode. The “.eeg” and “.eg2” files contain the data from the primary and secondary EEG channels, if these have been enabled. Very simply, the data consist of “num_EEG_samples” data bytes, following on from the data_start. The sample count is specified in the header. The “.egf” file is stored if a user selects a higher-sample rate EEG. Samples are normally collected at 4800 Hz (specified in the header), and are also normally 2 bytes long, rather than just 1.
```

In [19]:
# Hussaini lab tools: https://github.com/HussainiLab/BinConverter/blob/master/BinConverter/core/CreateEEG.py

from BinConverter.core.CreateEEG import (
    fir_hann, fir_hann, EEG_downsample, create_egf, write_eeg
)

Try using a NumpyRecordingExtractor to ingest '.eeg' or '.egf' data.
The user can ultimately specify whether to use .eeg or .egf data to be saved as LFP data in the nwb file. By default we
will prefer .eeg data, since it has a lower sampling rate and is more memory efficient. In most cases researchers will
not want to look at high frequency oscillations in LFP data.

We can first extract the data of interest as a numpy ndarray, then create a NumpyRecordingExtractor for the nwb conversion.


*  Case 1: Extract numpy ndarray from `.bin` data (very high Fs)

In [39]:
recording = se.AxonaRecordingExtractor(filename=set_file)

In [41]:
# Filter the data to avoid aliasing based on desired sampling frequency. The nyquest theorem states we need 
# twice the sampling rate of the highest frequency we are interested in. Conversely, we want to filter out all frequencies
# higher than half our sampling rate. Since filters are not perfect and have a certain transition width until full
# attenuation, we are conservative and choose sampling_rate / 2.25 as our cut-off high frequency threshold.

import spiketoolkit as st

sampling_rate = 250  # .eeg=250, .egf=4800

recording_filt = st.preprocessing.bandpass_filter(recording, freq_min=0, freq_max=np.floor(sampling_rate / 2.25))

In [31]:
timeseries = RX_bin.get_traces()

In [32]:
timeseries

(16, 14438700)

In [ ]:
timeseries.shape

*  Case 2: Extract numpy ndarray from `.eeg` data (low Fs)

*  Case 3: Extract numpy ndarray from `.egf` data (high Fs)

In [23]:

RX = se.NumpyRecordingExtractor()

RX

Exception ignored in: <function BaseExtractor.__del__ at 0x7fc18fedd9d0>
Traceback (most recent call last):
  File "/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/spikeextractors/baseextractor.py", line 36, in __del__
    for memmap_obj in self._memmap_files:
AttributeError: 'NumpyRecordingExtractor' object has no attribute '_memmap_files'


TypeError: __init__() missing 2 required positional arguments: 'timeseries' and 'sampling_frequency'

In [47]:
from nwb_conversion_tools.datainterfaces.ecephys.baselfpextractorinterface import BaseLFPExtractorInterface
from nwb_conversion_tools.datainterfaces.ecephys.axona.axonadatainterface import AxonaRecordingExtractorInterface
import spiketoolkit as st

class AxonaLFPInterface(BaseLFPExtractorInterface):
    """Primary data interface class for converting Axona LFP data."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = get_schema_from_method_signature(se.example_datasets.toy_example)
        source_schema['additionalProperties'] = True
        return source_schema

    def __init__(self, filename: str):
        super().__init__(filename=filename)
        sampling_rate = 250  # .eeg=250, .egf=4800
        self.recording_extractor = st.preprocessing.bandpass_filter(
            self.RX(filename=filename),
            freq_min=0,
            freq_max=np.floor(sampling_rate / 2.25)
        )
        self.subset_channels = None
        self.source_data = source_data

    def get_metadata(self):
        """Retrieve Ecephys metadata specific to the Axona format."""
        metadata = AxonaRecordingExtractorInterface(filename=set_filename).get_metadata()
        metadata['Ecephys'].update(
            ElectricalSeries_lfp=dict(
                name="LFP",
                description="Local field potential signal."
            )
        )

        return metadata

In [48]:
set_filename = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/new_session_data/06172021-HPC-B6-RAW/06172021-HPC-B6-RAW.set'

axo_lfp = AxonaLFPInterface(filename=set_filename)

In [50]:
axo_lfp.get_source_schema()

{'required': [],
 'properties': {'duration': {'type': 'number', 'default': 10.0},
  'num_channels': {'type': 'number', 'default': 4},
  'sampling_frequency': {'type': 'number', 'default': 30000.0},
  'K': {'type': 'number', 'default': 10},
  'dumpable': {'type': 'boolean', 'default': False},
  'dump_folder': {'type': 'string'},
  'seed': {'type': 'number'}},
 'type': 'object',
 'additionalProperties': True}

In [52]:
axo_lfp.get_conversion_options()

{}

In [53]:
axo_lfp.get_conversion_options_schema()

{'required': [],
 'properties': {'stub_test': {'type': 'boolean', 'default': False},
  'use_times': {'type': 'boolean', 'default': False},
  'save_path': {'type': 'string'},
  'overwrite': {'type': 'boolean', 'default': False},
  'buffer_mb': {'type': 'number', 'default': 500}},
 'type': 'object',
 'additionalProperties': False}

In [56]:
axo_lfp.run_conversion?

In [57]:
axo_lfp.run_conversion?

In [60]:
metadata

{'Ecephys': {'Device': [{'name': 'Axona',
    'description': 'Axona DacqUSB, sw_version=1.2.2.16',
    'manufacturer': 'Axona'}],
  'ElectrodeGroup': [{'name': 'Group0',
    'location': '',
    'device': 'Axona',
    'description': 'Group 0 electrodes.'},
   {'name': 'Group1',
    'location': '',
    'device': 'Axona',
    'description': 'Group 1 electrodes.'},
   {'name': 'Group2',
    'location': '',
    'device': 'Axona',
    'description': 'Group 2 electrodes.'},
   {'name': 'Group3',
    'location': '',
    'device': 'Axona',
    'description': 'Group 3 electrodes.'}],
  'Electrodes': [{'name': 'group_name',
    'description': 'The name of the ElectrodeGroup this electrode is a part of.'}],
  'ElectricalSeries_raw': {'name': 'ElectricalSeries_raw',
   'description': 'Raw acquisition traces.'},
  'ElectricalSeries_lfp': {'name': 'LFP',
   'description': 'Local field potential signal.'}},
 'NWBFile': {'session_start_time': '2021-06-17T12:22:02',
  'session_description': '',
  'exper

In [64]:
import uuid
from datetime import datetime
import warnings
import numpy as np
import distutils.version
from pathlib import Path
from typing import Union, Optional, List
from warnings import warn
import psutil
from collections import defaultdict

import spikeextractors as se
import pynwb
from numbers import Real
from hdmf.data_utils import DataChunkIterator
from hdmf.backends.hdf5.h5_utils import H5DataIO
#from .json_schema import dict_deep_update

PathType = Union[str, Path, None]
ArrayType = Union[list, np.ndarray]

In [66]:
nwbfile

root pynwb.file.NWBFile at 0x140464145403808
Fields:
  devices: {
    Axona <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Group0 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group1 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Group3 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  experimenter: ['Abid']
  file_create_date: [datetime.datetime(2021, 7, 27, 17, 43, 29, 372771, tzinfo=tzlocal())]
  identifier: 6aa5c17f-0e7e-4fdf-b7a5-d5c4d77a5841
  session_start_time: 2021-06-17 12:22:02+02:00
  timestamps_reference_time: 2021-06-17 12:22:02+02:00

In [77]:
from nwb_conversion_tools.utils.spike_interface import add_electrode_groups

In [79]:
axo_raw = AxonaRecordingExtractorInterface(filename=filename)

In [81]:
axo_lfp.recording_extractor.get_channel_property_names(channel_id=0)

['gain', 'group', 'name', 'offset']

In [80]:
recording = axo_lfp.recording_extractor
recording = axo_raw.recording_extractor
exclude = ()
nwbfile = make_nwbfile_from_metadata(metadata=metadata)


if nwbfile.electrodes is not None:
    ids_absent = [id not in nwbfile.electrodes.id for id in recording.get_channel_ids()]
    if not all(ids_absent):
        warnings.warn('cannot create electrodes for this recording as ids already exist')

if nwbfile is not None:
    assert isinstance(nwbfile, pynwb.NWBFile), "'nwbfile' should be of type pynwb.NWBFile"
if nwbfile.electrode_groups is None or len(nwbfile.electrode_groups) == 0:
    add_electrode_groups(recording, nwbfile, metadata)
# For older versions of pynwb, we need to manually add these columns
if distutils.version.LooseVersion(pynwb.__version__) < '1.3.0':
    if nwbfile.electrodes is None or 'rel_x' not in nwbfile.electrodes.colnames:
        nwbfile.add_electrode_column('rel_x', 'x position of electrode in electrode group')
    if nwbfile.electrodes is None or 'rel_y' not in nwbfile.electrodes.colnames:
        nwbfile.add_electrode_column('rel_y', 'y position of electrode in electrode group')

defaults = dict(
    x=np.nan,
    y=np.nan,
    z=np.nan,
    # There doesn't seem to be a canonical default for impedence, if missing.
    # The NwbRecordingExtractor follows the -1.0 convention, other scripts sometimes use np.nan
    imp=-1.0,
    location="unknown",
    filtering="none",
    group_name="0"
)
if metadata is None:
    metadata = dict(Ecephys=dict())

if 'Ecephys' not in metadata:
    metadata['Ecephys'] = dict()

if 'Electrodes' not in metadata['Ecephys']:
    metadata['Ecephys']['Electrodes'] = []

assert all([isinstance(x, dict) and set(x.keys()) == set(['name', 'description'])
            for x in metadata['Ecephys']['Electrodes']]), \
    "Expected metadata['Ecephys']['Electrodes'] to be a list of dictionaries, containing the keys 'name' and 'description'"
assert all([x['name'] != 'group' for x in metadata['Ecephys']['Electrodes']]), \
    "Passing metadata field 'group' is deprecated; pass group_name instead!"

if nwbfile.electrodes is None:
    nwb_elec_ids = []
else:
    nwb_elec_ids = nwbfile.electrodes.id.data[:]

elec_columns = defaultdict(dict)  # dict(name: dict(description='',data=data, index=False))
elec_columns_append = defaultdict(dict)
property_names = set()
for chan_id in recording.get_channel_ids():
    for i in recording.get_channel_property_names(channel_id=chan_id):
        property_names.add(i)

# property 'brain_area' of RX channels corresponds to 'location' of NWB electrodes
exclude_names = set(['location','group'] + list(exclude))

channel_property_defaults = {
    list: [],
    np.ndarray: np.array(np.nan),
    str: '',
    Real: np.nan
}
found_property_types = {prop: Real for prop in property_names}

for prop in property_names:
    prop_skip = False
    if prop not in exclude_names:
        data = []
        prop_chan_count = 0
        # build data:
        for chan_id in recording.get_channel_ids():
            if prop in recording.get_channel_property_names(channel_id=chan_id):
                prop_chan_count += 1
                chan_data = recording.get_channel_property(channel_id=chan_id, property_name=prop)
                # find the type and store (only when the first channel with given property is found):
                if prop_chan_count==1:
                    proptype = [proptype for proptype in channel_property_defaults if isinstance(chan_data, proptype)]
                    if len(proptype) > 0:
                        found_property_types[prop] = proptype[0]
                        # cast as float if any number:
                        if found_property_types[prop]==Real:
                            chan_data = np.float(chan_data)
                        # update data if wrong datatype items filled prior:
                        if len(data) > 0 and not isinstance(data[-1], found_property_types[prop]):
                            data = [channel_property_defaults[found_property_types[prop]]] * len(data)
                    else:
                        prop_skip = True  # skip storing that property if not of default type
                        break
                data.append(chan_data)
            else:
                data.append(channel_property_defaults[found_property_types[prop]])
        # store data after build:
        if not prop_skip:
            index = found_property_types[prop] == ArrayType
            prop_name_new = 'location' if prop == 'brain_area' else prop
            found_property_types[prop_name_new] = found_property_types.pop(prop)
            elec_columns[prop_name_new].update(description=prop_name_new, data=data, index=index)

for x in metadata['Ecephys']['Electrodes']:
    elec_columns[x['name']]['description'] = x['description']
    if x['name'] not in list(elec_columns):
        raise ValueError(f'"{x["name"]}" not a property of se object')

# updating default arguments if electrodes table already present:
default_updated = dict()
if nwbfile.electrodes is not None:
    for colname in nwbfile.electrodes.colnames:
        if colname!='group':
            samp_data = nwbfile.electrodes[colname].data[0]
            default_datatype = [proptype for proptype in channel_property_defaults if isinstance(samp_data, proptype)][0]
            default_updated.update({colname:channel_property_defaults[default_datatype]})
default_updated.update(defaults)

for name, des_dict in elec_columns.items():
    des_args = dict(des_dict)
    if name not in default_updated:
        if nwbfile.electrodes is None:
            nwbfile.add_electrode_column(name=name, description=des_args['description'], index=des_args['index'])
        else:
            # build default junk values for data to force add columns later:
            combine_data = [channel_property_defaults[found_property_types[name]]] * len(nwbfile.electrodes.id)
            des_args['data'] = combine_data + des_args['data']
            elec_columns_append[name] = des_args

for name in elec_columns_append:
    _ = elec_columns.pop(name)

for j, channel_id in enumerate(recording.get_channel_ids()):
    if channel_id not in nwb_elec_ids:
        electrode_kwargs = dict(default_updated)
        electrode_kwargs.update(id=channel_id)

        # recording.get_channel_locations defaults to np.nan if there are none
        location = recording.get_channel_locations(channel_ids=channel_id)[0]
        if all([not np.isnan(loc) for loc in location]):
            # property 'location' of RX channels corresponds to rel_x and rel_ y of NWB electrodes
            electrode_kwargs.update(
                dict(
                    rel_x=float(location[0]),
                    rel_y=float(location[1])
                )
            )

        for name, desc in elec_columns.items():
            if name == 'group_name':
                group_name = str(desc['data'][j])
                if group_name!='' and group_name not in nwbfile.electrode_groups:
                    warnings.warn(f"Electrode group {group_name} for electrode {channel_id} was not "
                                  "found in the nwbfile! Automatically adding.")
                    missing_group_metadata = dict(
                        Ecephys=dict(
                            ElectrodeGroup=[dict(
                                name=group_name,
                            )]
                        )
                    )
                    add_electrode_groups(recording, nwbfile, missing_group_metadata)
                electrode_kwargs.update(
                    dict(
                        group=nwbfile.electrode_groups[group_name],
                        group_name=group_name
                    )
                )
            elif 'data' in desc:
                electrode_kwargs[name] = desc['data'][j]

        if 'group_name' not in elec_columns:
            group_id = recording.get_channel_groups(channel_ids=channel_id)[0]
            electrode_kwargs.update(
                dict(
                    group=nwbfile.electrode_groups[str(group_id)],
                    group_name=str(group_id)
                )
            )

        nwbfile.add_electrode(**electrode_kwargs)
# add columns for existing electrodes:
for col_name, cols_args in elec_columns_append.items():
    nwbfile.add_electrode_column(col_name,**cols_args)
assert nwbfile.electrodes is not None, \
    "Unable to form electrode table! Check device, electrode group, and electrode metadata."

/tmp/ipykernel_2560/1069478143.py:89: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  chan_data = np.float(chan_data)


KeyError: 'data'

In [85]:
elec_columns

defaultdict(dict,
            {'offset': {'description': 'offset',
              'data': [0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0,
               0.0],
              'index': False},
             'group_name': {'description': 'The name of the ElectrodeGroup this electrode is a part of.'}})

In [74]:
nwbfile.electrodes[colname].data

[]

In [59]:
metadata = axo_lfp.get_metadata()

nwbfile = make_nwbfile_from_metadata(metadata=metadata)

axo_lfp.run_conversion(nwbfile, metadata)

/home/sbuergers/spikeinterface/hussaini-test-pipeline/venv/lib/python3.8/site-packages/pynwb/file.py:801: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


KeyError: 'data'

In [58]:
axo_lfp.run_conversion(
    metadata=None, 
    nwbfile=output_file,
    overwrite=True,
    save_path=output_file,
    conversion_options=None
)

TypeError: run_conversion() got an unexpected keyword argument 'conversion_options'

# Intan to NWB

For this there are already resources from the Hussaini lab we should be able to use.

See here: ...

In [39]:
from nwb_conversion_tools import (
    NWBConverter, AxonaRecordingExtractorInterface, AxonaPositionDataInterface, IntanRecordingInterface
)

In [40]:
from nwb_conversion_tools import IntanRecordingInterface

In [41]:
class HussainiIntanNWBConverter(NWBConverter):
    data_interface_classes = dict(
        IntanRecordingInterface=IntanRecordingInterface,
    )

In [42]:
intan_file = '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Intan_data/intan_rhd_test_1.rhd'

In [43]:
# Specify source data

source_data = dict(
    IntanRecordingInterface=dict(
        file_path=intan_file
    )
)
print(json.dumps(source_data, indent=2))

{
  "IntanRecordingInterface": {
    "file_path": "/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/Intan_data/intan_rhd_test_1.rhd"
  }
}


In [44]:
# Initialize HussainiIntanNWBConverter

intan_converter = HussainiIntanNWBConverter(source_data=source_data)

Source data is valid!


In [45]:
# Get metadata_schema from converter

metadata_schema = intan_converter.get_metadata_schema()

print(json.dumps(metadata_schema['properties'], indent=2))

{
  "NWBFile": {
    "required": [
      "session_description",
      "identifier",
      "session_start_time"
    ],
    "properties": {
      "session_description": {
        "type": "string",
        "format": "long",
        "description": "a description of the session where this data was generated",
        "default": "no description"
      },
      "identifier": {
        "type": "string",
        "description": "a unique text identifier for the file",
        "default": "74a1b10c-84a8-4a5f-8a2d-9ea071e282b4"
      },
      "session_start_time": {
        "type": "string",
        "description": "the start date and time of the recording session",
        "format": "date-time",
        "default": "1970-01-01T00:00:00"
      },
      "experimenter": {
        "type": "array",
        "items": {
          "type": "string",
          "title": "experimenter"
        },
        "description": "name of person who performed experiment"
      },
      "experiment_description": {
        "

In [46]:
# Validate metadata against metadata_schema

validate(
    instance=intan_converter.get_metadata(),
    schema=intan_converter.get_metadata_schema()
)

ValidationError: Additional properties are not allowed ('ElectricalSeries_raw' was unexpected)

Failed validating 'additionalProperties' in schema['properties']['Ecephys']:
    {'additionalProperties': False,
     'properties': {'Device': {'default': [{}],
                               'items': {'$ref': '#/properties/Ecephys/properties/definitions/Device'},
                               'minItems': 1,
                               'type': 'array'},
                    'ElectrodeGroup': {'default': [{'description': 'Group '
                                                                   'B '
                                                                   'electrodes.',
                                                    'name': 'GroupB'},
                                                   {'description': 'Group '
                                                                   'A '
                                                                   'electrodes.',
             